In [1]:
import copy
import itertools as its
import math
import os
import pathlib
import re
from typing import Dict, List, Optional
from collections import Counter, defaultdict

import networkx as nx
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from aoc import sim_new as sim, testing, util

twopi = 2 * math.pi

%matplotlib inline

INPUT_PATH = pathlib.Path('..') / 'input' / 'dec12.txt'

In [2]:
def read_pos(s: str) -> List[int]:
    return [int(x) for x in re.findall(r'<x=(-?\d+), y=(-?\d+), z=(-?\d+)', s)[0]]

testing.assert_all_equal(read_pos('<x=1, y=-4, z=10>'), [1, -4, 10])

In [3]:
def simulate(moons, steps, verbose: bool = False):
    moons = copy.copy(moons)
    moon_vs = [[0] * len(moon) for moon in moons]
    seen = set([str(moons) + str(moon_vs)])  # Start with initial position in seen
    step = 0
    while True:
        step += 1
        for i in range(len(moons)):
            left_moon = moons[i]
            left_moon_v = moon_vs[i]
            for j in range(i + 1, len(moons)):
                right_moon = moons[j]
                right_moon_v = moon_vs[j]
                for k in range(len(left_moon)):
                    if left_moon[k] < right_moon[k]:
                        left_moon_v[k] += 1
                        right_moon_v[k] -= 1
                    elif left_moon[k] > right_moon[k]:
                        left_moon_v[k] -= 1
                        right_moon_v[k] += 1
                    # Nothing if same
        for moon, moon_v in zip(moons, moon_vs):
            for i in range(len(moon)):
                moon[i] += moon_v[i]
        if step == steps:
            break
        if steps > 0:
            continue
        h = str(moons) + str(moon_vs)
        if verbose:
            print(step, h)
        if h in seen:
            return step
        seen.add(h)
        
    return sum(sum(abs(x) for x in moon) * sum(abs(x) for x in moon_v) for moon, moon_v in zip(moons, moon_vs))

In [4]:
moons = [read_pos(line) for line in INPUT_PATH.read_text().strip().split('\n')]
simulate(moons, 1000)

10635

In [5]:
moons = [read_pos(line) for line in INPUT_PATH.read_text().strip().split('\n')]
# This takes too long. So simulate each coordinate separately and find the LCM
# simulate(moons, -1)

In [6]:
util.lcm(
    simulate([[moon[0]] for moon in moons], -1),  # x axis
    util.lcm(
        simulate([[moon[1]] for moon in moons], -1),  # y axis
        simulate([[moon[2]] for moon in moons], -1)   # z axis
    )
)

583523031727256